<a href="https://colab.research.google.com/github/jhonKifle/NN_Regularization/blob/main/NN_Regularized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MNIST classification using Regularization (L2)**

In this demo, the previous classification is used to show for enhancing Overfitting of the model. Overfitting is a common problem in machine learning where a model learns the training data too well, capturing not only the underlying patterns but also noise and irrelevant details. As a result, the model performs well on the training data but fails to generalize to new, unseen data.

L2 regularization helps prevent overfitting by adding a penalty term to the loss function that penalizes large weights.

When L2 regularization is applied, the gradient descent algorithm updates the weights not only based on the original loss but also by including a term that shrinks the weights. The parameter update rule with L2 regularization becomes:

The weight update for NN with no regularizations was as follows:

 $$
 w′=w−η\frac{∂C_0}{∂w} \tag{1}
 $$
 $$
b'= b−η\frac{∂C0}{∂b} \tag{2}
$$
Introducing the regularizations (L2), the learning rule for the weights becomes:
 $$
 w′=w(1−\frac{ηλ}{n})−η\frac{∂C_0}{∂w} \tag{3}
 $$
Biases are typically not regularized, but weights are, as biases control the overall position of the function, while weights control its shape and complexity.

In [2]:
import numpy as np
import random
import pandas as pd

In [3]:
#Hot-encodding for output
def one_hot(y):
    one_hot_y = [np.zeros((10,1)) for a in y]
    for i,e in zip(y,one_hot_y):
      e[i] = 1
    return one_hot_y
#Dataset Preprocesing
def preprocess():
  #load dataset
  train = pd.read_csv('/content/sample_data/mnist_train_small.csv')
  test = pd.read_csv('/content/sample_data/mnist_test.csv')

  #NB size of train, & test dataset in this case is 19999 and 9999 respectively.
  #To make is fit to a size of batch of 10, I duplicate the first row as following 2 line of codes.
  train = train._append(train.head(1))
  test = test._append(test.head(1))

  valid = train.sample(frac=0.2)
  train = train.drop(valid.index)

  train_X = train.iloc[:,1:]
  train_y = train.iloc[:,0]

  train_X = train_X/255
  train_X = [np.reshape(x, (784, 1)) for x in train_X.values]

  valid_X = valid.iloc[:,1:]
  valid_X = valid_X/255
  valid_X = [np.reshape(x, (784, 1)) for x in valid_X.values]

  valid_y = valid.iloc[:,0]

  test_X = test.iloc[:,1:]
  test_X = test_X/255
  test_X = [np.reshape(x, (784, 1)) for x in test_X.values]
  test_y = test.iloc[:,0]

  train_y = train_y.to_numpy()
  train_y = one_hot(train_y)
  valid_y = valid_y.to_numpy()
  test_y = test_y.to_numpy()
  return list(zip(train_X,train_y)),list(zip(valid_X,valid_y)),list(zip(test_X,test_y))

In [4]:
class Cost:
  @staticmethod
  def Entropy(a,y):
    return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
  @staticmethod
  def delta(a,y):
    return (a-y)
class NN:
    def __init__(self, sizes,cost=Cost):
        self.layers = len(sizes)
        self.sizes = sizes
        self.cost = cost
        self.weightInitializer()

    # weights are inialized with gaussian variable of mean = 0 and sd = 1/sqrt(inputs)
    def weightInitializer(self):
        self.weights = [np.random.randn(y,x)   for x,y in zip(self.sizes[:-1],self.sizes[1:])]
        self.biases = [np.random.randn(y,1) for y in self.sizes[1:]]
    def feedforward(self,a):
        for w,b in zip(self.weights,self.biases):
            a = self.sigmoid(np.dot(w,a)+b)
        return a
    def sigmoid(self,z):
        return 1.0/(1.0+np.exp(-z))
    def sigmoid_prime(self,z):
        return self.sigmoid(z)*(1-self.sigmoid(z))
    def backprop(self,x,y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
          z = np.dot(w, activation)+b
          zs.append(z)
          activation = self.sigmoid(z)
          activations.append(activation)


        delta = (self.cost).delta(activations[-1],y) * self.sigmoid_prime(zs[-1])

        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2,self.layers):
            z = zs[-l]
            sp = self.sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(),delta)*sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta,activations[-l-1].transpose())
        return (nabla_b, nabla_w)
    def update_mini_batch(self,mini_batch,eta,lmbda,n):
      nabla_b = [np.zeros(b.shape) for b in self.biases]
      nabla_w = [np.zeros(w.shape) for w in self.weights]
      for x, y in mini_batch:
        delta_nabla_b, delta_nabla_w = self.backprop(x, y)
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
      self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
      self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]
    #Stochastic gradient Decsent with regularization (L2) - Lambda = 5.0
    def SGD(self,training_data,epochs,mini_batch_size,eta,lmbda=5.0,test_data=None):
      if test_data:
        n_test = len(test_data)
      for j in range(epochs):
        random.shuffle(training_data)
        for x in range(0,len(training_data),mini_batch_size):
          self.update_mini_batch(training_data[x:x+mini_batch_size],eta,lmbda,len(training_data))
        if test_data:
          err = self.evaluate(test_data)
          print("Epoch {} : {} / {} ({}%)".format(j,err,n_test,100*(err/n_test)))
        else:
          print("Epoch {} complete".format(j))
    def evaluate(self,test_data):
      test_results = [(np.argmax(self.feedforward(x)),y) for (x,y) in test_data]
      return sum(int(x==y) for (x,y) in test_results)


net = NN([784,30,10])
training_set, valid_set, test_set = preprocess()
net.SGD(training_set, 30, 10, 3.0, test_data=valid_set)



Epoch 0 : 3588 / 4000 (89.7%)
Epoch 1 : 3560 / 4000 (89.0%)
Epoch 2 : 3665 / 4000 (91.625%)
Epoch 3 : 3659 / 4000 (91.475%)
Epoch 4 : 3658 / 4000 (91.45%)
Epoch 5 : 3654 / 4000 (91.35%)
Epoch 6 : 3660 / 4000 (91.5%)
Epoch 7 : 3677 / 4000 (91.925%)
Epoch 8 : 3674 / 4000 (91.85%)
Epoch 9 : 3623 / 4000 (90.575%)
Epoch 10 : 3669 / 4000 (91.725%)
Epoch 11 : 3640 / 4000 (91.0%)
Epoch 12 : 3682 / 4000 (92.05%)
Epoch 13 : 3704 / 4000 (92.60000000000001%)
Epoch 14 : 3618 / 4000 (90.45%)
Epoch 15 : 3687 / 4000 (92.175%)
Epoch 16 : 3642 / 4000 (91.05%)
Epoch 17 : 3686 / 4000 (92.15%)
Epoch 18 : 3667 / 4000 (91.675%)
Epoch 19 : 3717 / 4000 (92.925%)
Epoch 20 : 3717 / 4000 (92.925%)
Epoch 21 : 3673 / 4000 (91.825%)
Epoch 22 : 3698 / 4000 (92.45%)
Epoch 23 : 3698 / 4000 (92.45%)
Epoch 24 : 3670 / 4000 (91.75%)
Epoch 25 : 3689 / 4000 (92.225%)
Epoch 26 : 3723 / 4000 (93.075%)
Epoch 27 : 3673 / 4000 (91.825%)
Epoch 28 : 3678 / 4000 (91.95%)
Epoch 29 : 3669 / 4000 (91.725%)
